In [2]:
!pip install ktrain

     --------------------------------------- 25.3/25.3 MB 59.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 6.9/6.9 MB 73.3 MB/s eta 0:00:00
     ---------------------------------------- 981.5/981.5 KB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 19.2/19.2 MB 73.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 115.2/115.2 KB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.8/2.8 MB 88.8 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 67.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started

You should consider upgrading via the 'c:\users\shyram\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import ktrain
from ktrain import text
import pandas as pd

**json 파일 다운로드**  
https://drive.google.com/uc?id=1-8urBLVtFuuvAVHi0s000e7r0KPUgt9f

In [6]:
df = pd.read_json(r'reviews_Digital_Music_5.json',lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5,Enya's last great album,1158019200,"09 12, 2006"
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5,Enya at her most elegant,991526400,"06 3, 2001"
2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5,The best so far,1058140800,"07 14, 2003"
3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5,Ireland produces good music.,957312000,"05 3, 2000"
4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4,4.5; music to dream to,1200528000,"01 17, 2008"


In [7]:
df = df[['reviewText','overall']]
df.head()

,reviewText,overall
0,"It's hard to believe ""Memory of Trees"" came ou...",5
1,"A clasically-styled and introverted album, Mem...",5
2,I never thought Enya would reach the sublime h...,5
3,This is the third review of an irish album I w...,5
4,"Enya, despite being a successful recording art...",4


In [8]:
sentiment = {1:'negative',2:'negative',3:'negative', 4:'positive',5:'positive'}

In [10]:
df['sentiment'] = df['overall'].map(sentiment)
df = df[['reviewText','sentiment']]
df.head()

,reviewText,sentiment
0,"It's hard to believe ""Memory of Trees"" came ou...",positive
1,"A clasically-styled and introverted album, Mem...",positive
2,I never thought Enya would reach the sublime h...,positive
3,This is the third review of an irish album I w...,positive
4,"Enya, despite being a successful recording art...",positive


In [11]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(train_df = df, 
                                                                   text_column = 'reviewText',
                                                                   label_columns=['sentiment'],
                                                                   maxlen=100, 
                                                                   max_features=100000,
                                                                   preprocess_mode='bert',
                                                                   val_pct=0.1)

['negative', 'positive']
       negative  positive
38430       0.0       1.0
12842       0.0       1.0
6965        0.0       1.0
8671        0.0       1.0
15755       0.0       1.0
['negative', 'positive']
       negative  positive
50214       0.0       1.0
53169       1.0       0.0
4119        0.0       1.0
37339       1.0       0.0
23168       1.0       0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [12]:
model = text.text_classifier(name='bert', train_data = (x_train, y_train) , preproc=preproc, metrics=['accuracy'])

Is Multi-Label? False
maxlen is 100
done.


In [13]:
learner = ktrain.get_learner(model = model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=32, 
                             use_multiprocessing = True)

In [14]:
learner.fit_onecycle(lr=2e-5, epochs=1,checkpoint_folder='output')



begin training using onecycle policy with max lr of 2e-05...
1820/1820 [==============================] - 368s 196ms/step - loss: 0.3300 - accuracy: 0.8638 - val_loss: 0.2699 - val_accuracy: 0.8873


In [15]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [16]:
predictor.predict('I loved the song')

'positive'

In [17]:
predictor.predict('I hated the song')

'negative'